In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
from itertools import count

In [ ]:
import json

In [ ]:
import re

# Get list of singers

In [ ]:
# fetch the url and create a beautiful soup object
def getSoup(url):
  response = requests.get(url)
  return BeautifulSoup(response.text, 'html.parser')


In [ ]:
# get links to singer's pages from given search result page
def getSingerLinksFromPage(url, page):
  url = f'{url}page/{page}'
  soup = getSoup(url)
  persons = soup.find_all('a', class_='person')
  return [person.attrs['href'] for person in persons]

In [ ]:
# get links to singer's pages from all search result pages
def getSingerLinks(url):
  all_links = []
  for i in count(1):
    links = getSingerLinksFromPage(url, i)
    if not links:
      break
    all_links.extend(links)
  return all_links

In [ ]:
# PeoplePill search result page for Sri Lankan Singers
url = 'https://peoplepill.com/browser/country/sri-lanka/occupation/singer/gender/all/years/all/'

In [ ]:
singer_links = getSingerLinks(url)

In [ ]:
singer_links

['/people/sanka-dineth/',
 '/people/wally-bastian/',
 '/people/gunadasa-kapuge/',
 '/people/amarasiri-peiris/',
 '/people/desmond-de-silva/',
 '/people/indrachapa-liyanage/',
 '/people/nalin-jayawardena/',
 '/people/sadiris-master/',
 '/people/rookantha-gunathilake/',
 '/people/sisira-senaratne/',
 '/people/shelton-premaratne/',
 '/people/narada-disasekara/',
 '/people/neela-wickramasinghe/',
 '/people/c-t-fernando/',
 '/people/gratien-ananda/',
 '/people/rukshan-perera/',
 '/people/priya-suriyasena/',
 '/people/rohan-shantha-bulegoda/',
 '/people/milton-mallawarachchi/',
 '/people/saheli-rochana-gamage/',
 '/people/saman-de-silva/',
 '/people/tharanga-goonetilleke/',
 '/people/yolande-bavan/',
 '/people/mercy-edirisinghe/',
 '/people/nihal-nelson/',
 '/people/dinesh-kanagaratnam/',
 '/people/keerthi-pasquel/',
 '/people/sunil-perera/',
 '/people/anthony-c-perera/',
 '/people/jagath-wickramasinghe/',
 '/people/lionel-ranwala/',
 '/people/clarence-wijewardena/',
 '/people/freddie-silva/

# Get Singer Info

In [ ]:
root_url = 'https://peoplepill.com'

In [ ]:
# extract info from singer's page
def getSingerInfo(url):
  soup = getSoup(root_url + url)
  res = {}

  res['name'] = soup.find('h1', class_='name').getText()
  res['info'] = soup.find('div', class_='brief-bio').getText()

  props = soup.select('tr.prop.border-bottom')
  for prop in props:
    label = prop.find(class_='label').getText().strip()
    print(f'\t{label}')
    value = prop.find(class_='values').getText().strip()
    res[label] = value
  
  body = soup.find('div', 'body html-content')
  if body:
    texts = body.find_all(['p', 'h1', 'h2', 'h3'], recursive=False)
    text = ''
    for i in texts:
      text += i.get_text(separator='\n')
    res['description'] = text

  return res

In [ ]:
getSingerInfo('/people/ranil-mallawarachchi/')

	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
	Residence
	Family


{'Age': '39 years',
 'Birth': '12 April 1982, Sri Lanka',
 'Family': 'Father:\n\nMilton Mallawarachchi',
 'From': 'Sri Lanka',
 'Gender': 'male',
 'Intro': 'Sri Lankan musician',
 'Is': 'Singer',
 'Residence': 'Narre Warren',
 'Type': 'Music',
 'info': 'Sri Lankan musician',
 'name': 'Ranil Mallawarachchi'}

In [ ]:
# extracting info from all singers
singers = []
for link in singer_links:
  print(link)
  info = getSingerInfo(link)
  singers.append(info)

/people/sanka-dineth/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
/people/wally-bastian/
	Intro
	Is
	From
	Type
	Gender
	Death
/people/gunadasa-kapuge/
	Intro
	Was
	From
	Type
	Gender
	Birth
	Death
	Star sign
/people/amarasiri-peiris/
	Intro
	Is
	From
	Type
	Gender
	Profiles
/people/desmond-de-silva/
	Intro
	Is
	From
	Type
	Gender
/people/indrachapa-liyanage/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
	Family
/people/nalin-jayawardena/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
/people/sadiris-master/
	Intro
	A.K.A.
	Is
	From
	Type
	Gender
	Birth
	Death
/people/rookantha-gunathilake/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
	Family
/people/sisira-senaratne/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
/people/shelton-premaratne/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
	Instruments:
/people/narada-disasekara/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Death
	Star sign
	Instruments:
/people/neela-wickramasinghe/
	Intro
	Is
	From
	Type
	Gender
	Birth
	Age
	Star sign
	Profiles
/people/c-t-fe

In [ ]:
singers

In [ ]:
# write to file
with open('singers_raw.json', 'w') as fout:
     fout.write(json.dumps(singers))

#Translate to Sinhala with google translate

In [ ]:
!pip install mtranslate

  Created wheel for mtranslate: filename=mtranslate-1.8-py3-none-any.whl size=3694 sha256=1b3c58209260a9e00725c0607f4a0cb5b4a89de5e6499c24a040f9f247207e4c
  Stored in directory: /root/.cache/pip/wheels/a4/5d/18/6ea549ba042f5f29728049065bdc8a5cdb3c88a413bb026c9a
Successfully built mtranslate


In [ ]:
from mtranslate import translate

In [ ]:
# load data
singers = []
with open('singers_raw.json', 'r') as fin:
     singers = json.loads(fin.read())

In [ ]:
# translate a single string to Sinhala
def translate_si(txt):
  lines = txt.split('\n')
  lst = ['']
  res = ''
  for i in lines:
    if len(lst[-1]) + len(i) < 5000:
      lst[-1] += '\n\n' + i
    else:
      lst.append('\n\n' + i)
  for i in lst:
    res += '\n\n' + translate(i, 'si', 'en')
  res = re.sub(r'\n\s*\n', '\n\n', res)
  return res[2:]


In [ ]:
# singers = [singers[0]]

In [ ]:
# translate all of Singers fields to Sinhala
singers_si = []
for i, singer in enumerate(singers):
  print(f'translating singer: {i}')
  singer_si = {}
  for k, v in singer.items():
    singer_si[k] = translate_si(v)
  singers_si.append(singer_si)

In [ ]:
# singers[31]['description']

In [ ]:
# translate_si(singers[31]['description'])

In [ ]:
# for singer in singers_si:
#   for k, v in singer.items():
#     print(k, v)

In [ ]:
# writing data to file
with open('singers_si_raw.json', 'w', encoding='utf-8') as fout:
     fout.write(json.dumps(singers_si, ensure_ascii=False))

# Clean data

In [ ]:
# convert field names to lower case

In [ ]:
singers_si = []
with open('singers_si.json', 'r') as fin:
     singers_si = json.loads(fin.read())

In [ ]:
singers_1 = []
for i, singer in enumerate(singers_si):
  singer_tmp = {}
  for k, v in singer.items():
    singer_tmp[k.lower()] =v
  singers_1.append(singer_tmp)

In [ ]:
with open('singers_si.json', 'w', encoding='utf-8') as fout:
     fout.write(json.dumps(singers_1, ensure_ascii=False))

In [ ]:
from collections import defaultdict

In [ ]:
# list all the fields 
stat = defaultdict(int)
for i, singer in enumerate(singers_1):
  for k, v in singer.items():
    stat[k] += 1
sorted_stat = sorted(stat.items(), key=lambda x: -x[1])
for k, v in sorted_stat:
  print(k.ljust(20), str(v).rjust(3))

print([k for k, v in sorted_stat])
fields = [k for k, v in sorted_stat[:10]]
print(fields)

name                  80
info                  80
from                  80
gender                80
intro                 79
description           78
type                  77
birth                 71
is                    54
age                   39
death                 31
was                   26
star sign             24
instruments:          17
profiles              13
family                10
a.k.a.                 7
education              5
known for              2
residence              2
genre:                 1
politics               1
stats                  1
notable works          1
['name', 'info', 'from', 'gender', 'intro', 'description', 'type', 'birth', 'is', 'age', 'death', 'was', 'star sign', 'instruments:', 'profiles', 'family', 'a.k.a.', 'education', 'known for', 'residence', 'genre:', 'politics', 'stats', 'notable works']
['name', 'info', 'from', 'gender', 'intro', 'description', 'type', 'birth', 'is', 'age']


In [ ]:
# singers_2 = []
# for i, singer in enumerate(singers_1):
#   singer_tmp = {}
#   for k, v in singer.items():
#     if k in fields:
#       singer_tmp[k] =v
#   singers_2.append(singer_tmp)